In [ ]:
import sys, os
from pathlib import Path
import matplotlib.pyplot as plt

from shutil import rmtree
import vtk
import numpy as np
import matplotlib.pyplot as plt

root_path = os.path.join(Path().resolve().parent.parent)
package_path = os.path.join(root_path, "hamageolib")
result_path = os.path.join(Path().resolve(), "results")

if str(package_path) not in sys.path:
    sys.path.insert(0, str(package_path))

if not os.path.isdir(result_path):
    os.mkdir(result_path)

In [ ]:
case_path = os.path.join(Path().resolve(), "../../big_tests/TwoDSubduction/EBA_CDPT_test_perplex_mixing_log")

assert(os.path.isdir(case_path))

file_path = os.path.join(case_path, "output", "solution", "solution-00104.pvtu")

assert(os.path.isfile(file_path))

### Read the pvtu file and process the inputs as grid

In [ ]:
# Read the PVTU file as an Unstructured Grid
reader = vtk.vtkXMLPUnstructuredGridReader()
reader.SetFileName(file_path)
reader.Update()
u_grid = reader.GetOutput()

# Convert Unstructured Grid to PolyData
geometry_filter = vtk.vtkGeometryFilter()
geometry_filter.SetInputData(reader.GetOutput())
geometry_filter.Update()
poly_data = geometry_filter.GetOutput()

### Print Mesh Information

In [ ]:
from utils.vtk_utilities import estimate_memory_usage

# Get the number of points and cells in u_grid (full unstructured grid)
num_points_u = u_grid.GetNumberOfPoints()
num_cells_u = u_grid.GetNumberOfCells()

# Get the number of points and cells in poly_data (surface representation)
num_points_poly = poly_data.GetNumberOfPoints()
num_cells_poly = poly_data.GetNumberOfCells()

# Print results
print("Unstructured Grid (Full Volume Mesh):")
print(f"  - Number of Points: {num_points_u}")
print(f"  - Number of Cells: {num_cells_u}")

print("\nPolyData (Extracted Surface Mesh):")
print(f"  - Number of Points: {num_points_poly}")
print(f"  - Number of Cells: {num_cells_poly}")

# Example usage
# Assuming u_grid is already loaded
memory_usage_mb = estimate_memory_usage(u_grid)
print(f"\nEstimated Memory Usage: {memory_usage_mb:.2f} MB")

### VTK Composition Extraction Function

This function, `vtk_extract_comp`, processes a **VTK unstructured grid (`u_grid`)** to extract a composition-defined surface by summing multiple composition markers. It applies a **threshold filter** to retain only regions where the summed composition values exceed a specified threshold.

#### **Constants and Assumptions**
- **Threshold Default**: \( 0.5 \) (Only points with summed composition **≥ 0.5** are retained).
- **Summation Approach**: If multiple fields are specified, their **sum defines the composition shape**.
- **Strict Field Checking**: If any field is missing, the function **raises an assertion error**.

#### **Output**
- **Returns**: `vtk.vtkPolyData` representing the extracted composition-based surface.
- **Failure Cases**: Returns `None` if required fields are not found.

In [ ]:
from utils.vtk_utilities  import vtk_extract_comp

# todo_slab
# Example Usage
slab_polydata = vtk_extract_comp(u_grid, "spcrust", "spharz")

if slab_polydata:
    print(f"Extracted Slab Shape: {slab_polydata.GetNumberOfPoints()} points, {slab_polydata.GetNumberOfCells()} cells.")


Extract the outline of the slab

### **Extracting Slab Outline with Depth Binning**
#### **Function Overview**
The function `extract_slab_outline_binned_unified` extracts the slab outline by applying depth binning and converting coordinates to a unified system. The extracted outline consists of lateral extrema within each depth bin.

#### **Constants and Assumptions**
- Default depth bin size: **10 km**.
- Input coordinates may be either Cartesian or spherical.
- Slab outline is defined by lateral extrema within depth bins.

#### **Output**
- `vtkOutlinePolydata_0, vtkOutlinePolydata_1`: Two `vtkPolyData` objects containing the slab outline points.

In [ ]:
from research.haoyuan_2d_subduction.post_process  import extract_slab_outline_binned_unified

# Example Usage
slab_outline_binned_0, slab_outline_binned_1 = extract_slab_outline_binned_unified(slab_polydata, True, bin_size=10.0e3)

# Extract outline points
outline_points = slab_outline_binned_1.GetPoints()
num_outline_points = slab_outline_binned_1.GetNumberOfPoints()
outline_array = np.array([outline_points.GetPoint(i) for i in range(num_outline_points)])

# Extract X and Y coordinates for outline
outline_x, outline_y = outline_array[:, 0], outline_array[:, 1]

# Save to file with header
ofile_path = os.path.join(result_path, "outline_coordinates.txt")
np.savetxt(ofile_path, outline_array, 
           fmt="%.4e",  # Format with 6 decimal places
           header="# x (m)\n# y (m)\n# z (m)", 
           comments="")  # Prevents '#' before the header

print("Save file %s" % ofile_path)


Following this setup, we use Matplotlib to plot the layout of the slab. The internal structure of the slab is represented by scattered points. In the second plot, we zoom into a smaller region highlighted in the first plot to investigate the trench area.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Define zoom-in region (adjust as needed)
x_range = (4.8e6, 5.3e6)  # Example: X range to zoom in
y_range = (3.5e6, 4.0e6)  # Example: Y range to zoom in

# Extract points from slab_polydata
points = slab_polydata.GetPoints()
num_points = slab_polydata.GetNumberOfPoints()

# Convert VTK points to NumPy array
points_array = np.array([points.GetPoint(i) for i in range(num_points)])

# Extract X and Y coordinates
x, y = points_array[:, 0], points_array[:, 1]

# Create a mask for points inside the zoomed-in region
zoom_mask = (x >= x_range[0]) & (x <= x_range[1]) & (y >= y_range[0]) & (y <= y_range[1])
zoomed_points = points_array[zoom_mask]

# Create a mask for outline points inside the zoomed-in region
zoom_outline_mask = (outline_x >= x_range[0]) & (outline_x <= x_range[1]) & \
                    (outline_y >= y_range[0]) & (outline_y <= y_range[1])
zoomed_outline = outline_array[zoom_outline_mask]

# Plotting
fig, axs = plt.subplots(1, 2, figsize=(12, 6))

# Full slab plot
axs[0].scatter(x, y, s=1, color='black', label="All Points")
axs[0].plot(outline_x, outline_y, 'b-', linewidth=1.5, label="Outline")  # Outline in blue
axs[0].set_xlabel("X Coordinate")
axs[0].set_ylabel("Y Coordinate")
axs[0].set_title("Full Slab Shape")
axs[0].legend()
axs[0].grid(True)

# Highlight zoomed region
axs[0].plot([x_range[0], x_range[0], x_range[1], x_range[1], x_range[0]],
            [y_range[0], y_range[1], y_range[1], y_range[0], y_range[0]],
            'r-', linewidth=1.5, label="Zoom Region")

# Zoomed-in subplot
axs[1].scatter(zoomed_points[:, 0], zoomed_points[:, 1], s=5, color='red', label="Zoomed Points")
axs[1].plot(zoomed_outline[:, 0], zoomed_outline[:, 1], 'b-', linewidth=1.5, label="Zoomed Outline")  # Zoomed outline
axs[1].set_xlim(x_range)
axs[1].set_ylim(y_range)
axs[1].set_xlabel("X Coordinate (Zoomed)")
axs[1].set_ylabel("Y Coordinate (Zoomed)")
axs[1].set_title("Zoomed-In Slab Region")
axs[1].legend()
axs[1].grid(True)

plt.tight_layout()
plt.show()
